1) “cnsRoadEditor”
Создать классы с методами для генерации карты дорог.

In [ ]:
using System;
using System.IO;
using System.Text;
using System.Collections.Generic;

public enum RoadSymbol
{
    Horizontal = '─',
    Vertical = '│',
    Intersection = '┼',
    TJunctionDown = '┬',
    TJunctionUp = '┴',
    TJunctionRight = '├',
    TJunctionLeft = '┤',
    Empty = ' ',
    TopLeftCorner = '┌',
    TopRightCorner = '┐',
    BottomLeftCorner = '└',
    BottomRightCorner = '┘',
    EndUp = '╵',
    EndDown = '╷',
    EndLeft = '╴',
    EndRight = '╶',
    Isolated = 'o',
    Unknown = '?'
}

public class RoadMap
{
    private readonly bool[,] _grid;
    public int Rows => _grid.GetLength(0);
    public int Cols => _grid.GetLength(1);

    public RoadMap(int rows, int cols)
    {
        if (rows <= 0 || cols <= 0)
        {
            throw new ArgumentException("Размеры карты должны быть > 0.");
        }
        _grid = new bool[rows, cols];
    }

    private RoadMap(bool[,] grid)
    {
        _grid = grid;
    }

    public bool GetCell(int row, int col)
    {
        // Проверяем, что координаты находятся в пределах карты
        return (row >= 0 && row < Rows && col >= 0 && col < Cols) && _grid[row, col];
    }

    private void SetCell(int row, int col, bool value)
    {
        // Устанавливаем значение ячейки, если координаты корректны
        if (row >= 0 && row < Rows && col >= 0 && col < Cols)
        {
            _grid[row, col] = value;
        }
    }

    public void DrawHorizontalLine(int x1, int x2, int y)
    {
        for (int i = Math.Min(x1, x2); i <= Math.Max(x1, x2); i++)
        {
            SetCell(y, i, true);
        }
    }

    public void DrawVerticalLine(int y1, int y2, int x)
    {
        for (int i = Math.Min(y1, y2); i <= Math.Max(y1, y2); i++)
        {
            SetCell(i, x, true);
        }
    }

    public void DrawRectangle(int x1, int y1, int x2, int y2)
    {
        DrawHorizontalLine(x1, x2, y1);
        DrawHorizontalLine(x1, x2, y2);
        DrawVerticalLine(y1, y2, x1);
        DrawVerticalLine(y1, y2, x2);
    }

    public void Clear()
    {
        Array.Clear(_grid, 0, _grid.Length);
    }


    public void GenerateWalkableMap(int segments, int minLength, int maxLength)
    {
        Clear();
        Random random = new Random();
        int currentX = Cols / 2;
        int currentY = Rows / 2;

        for (int i = 0; i < segments; i++)
        {
            int direction = random.Next(4); // 0-Up, 1-Down, 2-Left, 3-Right
            int length = random.Next(minLength, maxLength + 1);

            for (int j = 0; j < length; j++)
            {
                SetCell(currentY, currentX, true);
                switch (direction)
                {
                    case 0: currentY--; break;
                    case 1: currentY++; break;
                    case 2: currentX--; break;
                    case 3: currentX++; break;
                }
                
                // Проверка на выход за пределы карты
                if (currentY < 1 || currentY >= Rows - 1 || currentX < 1 || currentX >= Cols - 1)
                {
                    currentY = Math.Clamp(currentY, 1, Rows - 2);
                    currentX = Math.Clamp(currentX, 1, Cols - 2);
                    break;
                }
            }
        }
    }

    public bool SaveToFile(string filePath)
    {
        try
        {
            using (var writer = new StreamWriter(filePath))
            {
                writer.WriteLine($"{Rows} {Cols}");
                for (int r = 0; r < Rows; r++)
                {
                    for (int c = 0; c < Cols; c++)
                    {
                        writer.Write(_grid[r, c] ? '1' : '0');
                    }
                    writer.WriteLine();
                }
            }
            return true;
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Ошибка сохранения: {ex.Message}");
            return false;
        }
    }

    public static RoadMap LoadFromFile(string filePath)
    {
        try
        {
            using (var reader = new StreamReader(filePath))
            {
                string[] dimensions = reader.ReadLine()?.Split(' ') ?? new string[0];
                if (dimensions.Length != 2 || !int.TryParse(dimensions[0], out int rows) || !int.TryParse(dimensions[1], out int cols))
                {
                    throw new InvalidDataException("Неверный формат размеров в файле.");
                }

                var grid = new bool[rows, cols];
                for (int i = 0; i < rows; i++)
                {
                    string line = reader.ReadLine();
                    for (int j = 0; j < cols; j++)
                    {
                        grid[i, j] = line[j] == '1';
                    }
                }
                return new RoadMap(grid);
            }
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Ошибка загрузки: {ex.Message}");
            return null;
        }
    }
}

public static class RoadMapRenderer
{
    /// <summary>
    /// Преобразует логическую карту (true/false) в карту символов (─, │, ┌ и т.д.).
    /// </summary>
    public static char[,] ToSymbolMap(RoadMap map)
    {
        var symbolMap = new char[map.Rows, map.Cols];
        for (int r = 0; r < map.Rows; r++)
        {
            for (int c = 0; c < map.Cols; c++)
            {
                if (!map.GetCell(r, c))
                {
                    symbolMap[r, c] = (char)RoadSymbol.Empty;
                    continue;
                }

                // Вычисляем код на основе соседей с помощью битовых масок
                int code = (map.GetCell(r - 1, c) ? 1 : 0) |  // Сверху
                           (map.GetCell(r + 1, c) ? 2 : 0) |  // Снизу
                           (map.GetCell(r, c - 1) ? 4 : 0) |  // Слева
                           (map.GetCell(r, c + 1) ? 8 : 0);   // Справа

                // Выбираем символ на основе вычисленного кода
                symbolMap[r, c] = code switch
                {
                    0 => (char)RoadSymbol.Isolated,         // .
                    1 => (char)RoadSymbol.EndDown,          // ╵
                    2 => (char)RoadSymbol.EndUp,            // ╷
                    3 => (char)RoadSymbol.Vertical,         // │
                    4 => (char)RoadSymbol.EndRight,         // ╴
                    5 => (char)RoadSymbol.BottomRightCorner,// ┘
                    6 => (char)RoadSymbol.TopRightCorner,   // ┐
                    7 => (char)RoadSymbol.TJunctionLeft,    // ┤
                    8 => (char)RoadSymbol.EndLeft,          // ╶
                    9 => (char)RoadSymbol.BottomLeftCorner, // └
                    10 => (char)RoadSymbol.TopLeftCorner,   // ┌
                    11 => (char)RoadSymbol.TJunctionRight,  // ├
                    12 => (char)RoadSymbol.Horizontal,      // ─
                    13 => (char)RoadSymbol.TJunctionUp,     // ┴
                    14 => (char)RoadSymbol.TJunctionDown,   // ┬
                    15 => (char)RoadSymbol.Intersection,    // ┼
                    _ => (char)RoadSymbol.Unknown           // ?
                };
            }
        }
        return symbolMap;
    }

    public static void PrintSymbolMapToConsole(char[,] symbolMap)
    {
        var sb = new StringBuilder();
        for (int r = 0; r < symbolMap.GetLength(0); r++)
        {
            for (int c = 0; c < symbolMap.GetLength(1); c++)
            {
                sb.Append(symbolMap[r, c]);
            }
            sb.AppendLine();
        }
        Console.Write(sb.ToString());
    }
}

public class SymbolSpritesMap
{
    private readonly char[,] _symbolMap;
    private readonly Dictionary<char, char[,]> _sprites;

    public SymbolSpritesMap(char[,] symbolMap, char spriteChar = '█')
    {
        _symbolMap = symbolMap;
        _sprites = CreateSpriteSet(spriteChar);
    }

    public void PrintToConsole()
    {
        var sb = new StringBuilder();
        int spriteHeight = 3;
        int spriteWidth = 5;

        // Внешний цикл по строкам карты
        for (int mapRow = 0; mapRow < _symbolMap.GetLength(0); mapRow++)
        {
            // Цикл по строкам внутри одного спрайта (0, 1, 2)
            for (int spriteRow = 0; spriteRow < spriteHeight; spriteRow++)
            {
                // Цикл по столбцам карты
                for (int mapCol = 0; mapCol < _symbolMap.GetLength(1); mapCol++)
                {
                    if (_sprites.TryGetValue(_symbolMap[mapRow, mapCol], out var sprite))
                    {
                        // Цикл по столбцам внутри одного спрайта
                        for (int spriteCol = 0; spriteCol < spriteWidth; spriteCol++)
                        {
                            sb.Append(sprite[spriteRow, spriteCol]);
                        }
                    }
                    else
                    {
                        // Если спрайт не найден, вставляем пустую область
                        sb.Append(' ', spriteWidth);
                    }
                }
                sb.AppendLine();
            }
        }
        Console.Write(sb.ToString());
    }

    private static Dictionary<char, char[,]> CreateSpriteSet(char s) => new Dictionary<char, char[,]>
    {
        [(char)RoadSymbol.Horizontal]      = new[,] { {' ',' ',' ',' ',' '}, {s,s,s,s,s}, {' ',' ',' ',' ',' '} },
        [(char)RoadSymbol.Vertical]        = new[,] { {' ',' ',s,' ',' '}, {' ',' ',s,' ',' '}, {' ',' ',s,' ',' '} },
        [(char)RoadSymbol.Intersection]    = new[,] { {' ',' ',s,' ',' '}, {s,s,s,s,s}, {' ',' ',s,' ',' '} },
        [(char)RoadSymbol.TJunctionDown]   = new[,] { {' ',' ',' ',' ',' '}, {s,s,s,s,s}, {' ',' ',s,' ',' '} },
        [(char)RoadSymbol.TJunctionUp]     = new[,] { {' ',' ',s,' ',' '}, {s,s,s,s,s}, {' ',' ',' ',' ',' '} },
        [(char)RoadSymbol.TJunctionRight]  = new[,] { {' ',' ',s,' ',' '}, {' ',' ',s,s,s}, {' ',' ',s,' ',' '} },
        [(char)RoadSymbol.TJunctionLeft]   = new[,] { {' ',' ',s,' ',' '}, {s,s,s,' ',' '}, {' ',' ',s,' ',' '} },
        [(char)RoadSymbol.TopLeftCorner]   = new[,] { {' ',' ',' ',' ',' '}, {' ',' ',s,s,s}, {' ',' ',s,' ',' '} },
        [(char)RoadSymbol.TopRightCorner]  = new[,] { {' ',' ',' ',' ',' '}, {s,s,s,' ',' '}, {' ',' ',s,' ',' '} },
        [(char)RoadSymbol.BottomLeftCorner]= new[,] { {' ',' ',s,' ',' '}, {' ',' ',s,s,s}, {' ',' ',' ',' ',' '} },
        [(char)RoadSymbol.BottomRightCorner]=new[,] { {' ',' ',s,' ',' '}, {s,s,s,' ',' '}, {' ',' ',' ',' ',' '} },
        [(char)RoadSymbol.Empty]           = new[,] { {' ',' ',' ',' ',' '}, {' ',' ',' ',' ',' '}, {' ',' ',' ',' ',' '} }
    };
}


In [19]:
// Создаем карту
var manualMap = new RoadMap(rows: 15, cols: 40);
// Рисуем на ней фигуры
manualMap.DrawRectangle(2, 2, 12, 7); // x1, y1, x2, y2
manualMap.DrawRectangle(15, 5, 25, 12);
manualMap.DrawHorizontalLine(10, 17, 9);

// Конвертируем в символы и выводим
Console.WriteLine("--- Карта в виде символов ---");
char[,] symbolMap = RoadMapRenderer.ToSymbolMap(manualMap);
RoadMapRenderer.PrintSymbolMapToConsole(symbolMap);

--- Карта в виде символов ---
                                        
                                        
  ┌─────────┐                           
  │         │                           
  │         │                           
  │         │  ┌─────────┐              
  │         │  │         │              
  └─────────┘  │         │              
               │         │              
          ╴────┼─╶       │              
               │         │              
               │         │              
               └─────────┘              
                                        
                                        


In [14]:
string filePath = "my_notebook_map.txt";
Console.WriteLine($"Попытка сохранить карту в '{filePath}'...");

if (manualMap.SaveToFile(filePath))
{
    Console.WriteLine("Сохранение успешно. Загружаем обратно...");
    RoadMap loadedMap = RoadMap.LoadFromFile(filePath);
    if (loadedMap != null)
    {
        Console.WriteLine("--- Загруженная карта ---");
        var loadedSymbolMap = RoadMapRenderer.ToSymbolMap(loadedMap);
        RoadMapRenderer.PrintSymbolMapToConsole(loadedSymbolMap);
        File.Delete(filePath); // Удаляем временный файл
        Console.WriteLine($"\nФайл '{filePath}' удален.");
    }
}

Попытка сохранить карту в 'my_notebook_map.txt'...
Сохранение успешно. Загружаем обратно...
--- Загруженная карта ---
                                        
                                        
  ┌─────────┐                           
  │         │                           
  │         │                           
  │         │  ┌─────────┐              
  │         │  │         │              
  └─────────┘  │         │              
               │         │              
          ╴────┼─╶       │              
               │         │              
               │         │              
               └─────────┘              
                                        
                                        

Файл 'my_notebook_map.txt' удален.


In [7]:
Console.WriteLine("Случайно сгенерированная карта");
var randomMap = new RoadMap(rows: 10, cols: 60);

// Вызываем новый, улучшенный метод
randomMap.GenerateWalkableMap(segments: 20, minLength: 3, maxLength: 8);

var randomSymbolMap = RoadMapRenderer.ToSymbolMap(randomMap);
RoadMapRenderer.PrintSymbolMapToConsole(randomSymbolMap);

Случайно сгенерированная карта
                                                            
 ┌──────────┬──────────┐                                    
 │          │          │                                    
 │          │          │                                    
 │          │          │                                    
 │          │          └──────╶                             
 │          │                                               
 │          │                                               
 └──────────┘                                               
                                                            


In [20]:
Console.WriteLine("Карта в виде 'спрайтов'");
var spriteRenderer = new SymbolSpritesMap(symbolMap, '█');
spriteRenderer.PrintToConsole();

Карта в виде 'спрайтов'
                                                                                                                                                                                                        
                                                                                                                                                                                                        
                                                                                                                                                                                                        
                                                                                                                                                                                                        
                                                                                                                                                                            